<a href="https://colab.research.google.com/github/girotodenis/ReconhecimentoImagemVisaoComputacional/blob/master/Projeto2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NOME COMPLETO DOS INTEGRANTES DO GRUPO:
**Davi Miranda Gonçalves**, 
**Denis SIlva Giroto**,
**Naiara Moura Pires**

MATRÍCULAS DOS INTEGRANTES DO GRUPO:
**1931143030**,
**1931143029**,
**1931143033**

In [1]:
!git clone -l -s https://github.com/girotodenis/ReconhecimentoImagemVisaoComputacional.git projeto
#%cd cloned-repo
!ls

Cloning into 'projeto'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 230 (delta 20), reused 9 (delta 1), pack-reused 184
Receiving objects: 100% (230/230), 97.83 MiB | 35.93 MiB/s, done.
Resolving deltas: 100% (99/99), done.
projeto  sample_data


In [2]:
!curl -LJO https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov3.weights --output yolov3.weights

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   638  100   638    0     0   4833      0 --:--:-- --:--:-- --:--:--  4833
100  236M  100  236M    0     0  88.1M      0  0:00:02  0:00:02 --:--:-- 95.4M
curl: Saved to filename 'yolov3.weights'


In [3]:
%ls 'projeto' 

Atividade1.ipynb  gravar_video.ipynb  Projeto3.ipynb  README.md   yolov4.cfg
Atividade2.ipynb  imagens/            Projeto4.ipynb  videos/
coco.names        Projeto2.ipynb      Projeto.ipynb   yolov3.cfg


In [4]:
from google.colab.patches import cv2_imshow

import numpy as np
import cv2
import imutils

import time 


In [5]:
net = cv2.dnn.readNet('yolov3.weights','./projeto/yolov3.cfg')

In [6]:
classes = []
with open('./projeto/coco.names','r')as f:
  classes = [line.strip() for line in f.readlines()]
print(classes)

layer_names = net.getLayerNames()
outputLayers = [layer_names[i[0]-1] for i in net.getUnconnectedOutLayers()]
print(outputLayers)

cores = np.random.uniform(0,255,size=(len(classes), 3))

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']
['yolo_82', 'yolo_94', 'yolo_106']


In [7]:
def encontrar_objetos_dnn(outs):
  class_ids = []
  confidences = []
  boxes = []

  for out in outs:
    for detection in out:
      scores = detection[5:]
      class_id = np.argmax(scores)
      confidence = scores[class_id]

      if confidence>0.6:
        x,y,w,h,cx,cy = calcula_box(detection,width,height)
        boxes.append([x,y,w,h,cx,cy])
        confidences.append(confidence)
        class_ids.append(class_id)
      
      # print(class_ids)

  return class_ids, confidences, boxes

In [8]:
def calcula_box(detection, width, height):
  center_x = int(detection[0]*width)
  center_y = int(detection[1]*height)
  w = int(detection[2]*width)
  h = int(detection[3]*height)
  x = int(center_x - w/2)
  y = int(center_y - h/2)
  return [x,y,w,h,center_x,center_y]

In [9]:
def desenha_box(frame, boxes, class_ids, confidences):
  for i in range(len(boxes)):
    x,y,w,h,cx,cy = boxes[i]
    label = str(classes[class_ids[i]])
    label += ' '+ str(round(confidences[i],2))
    cor = cores[class_ids[i]]
    cv2.rectangle(frame,(x ,y),(x+w, y+h),cor, 2)
    cv2.putText(frame, label, (x, y+30), cv2.FONT_HERSHEY_PLAIN,1,(255,255,255),2)

In [10]:
carros_detectados = []

h_bins = 50
s_bins = 60
histSize = [h_bins, s_bins]
# hue varies from 0 to 179, saturation from 0 to 255
h_ranges = [0, 180]
s_ranges = [0, 256]
ranges = h_ranges + s_ranges # concat lists
# Use the 0-th and 1-st channels
channels = [0, 1]

def registrar_carro(frame, box):
  x,y,w,h,cx,cy = box
  crop_box = frame[y:y+h, x:x+w]
  gray_crop_box = cv2.cvtColor(crop_box, cv2.COLOR_BGR2HSV) 
  exists = _is_carro_registrado(gray_crop_box)

  if (not exists) :  
    carros_detectados.append(gray_crop_box)

  return carros_detectados

def _is_carro_registrado(gray_crop_box):
  
  cropped_image_histogram = cv2.calcHist([gray_crop_box], channels, None, histSize, ranges, accumulate=False)

  if (len(carros_detectados) == 0):
    return False
  
  exists = False;

  for detected in carros_detectados:
    detected_hist = cv2.calcHist([detected], channels, None, histSize, ranges, accumulate=False)

    # https://docs.opencv.org/3.4/d8/dc8/tutorial_histogram_comparison.html
    # https://docs.opencv.org/4.1.2/d6/dc7/group__imgproc__hist.html#gaf4190090efa5c47cb367cf97a9a519bd
    # https://docs.opencv.org/4.1.2/d6/dc7/group__imgproc__hist.html#ga994f53817d621e2e4228fc646342d386
    a = cv2.compareHist(cropped_image_histogram, detected_hist, cv2.HISTCMP_CORREL)
    
    if (a > 0.60):
      exists = True
      break 

  return exists

In [ ]:
cap = cv2.VideoCapture('./projeto/videos/Car2165.mp4')
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
writer = cv2.VideoWriter('./projeto/videos/novo_Car2165.mp4', cv2.VideoWriter_fourcc(*'DIVX'),20,(width, height))

print(width, height)
if cap.isOpened() == False:
    print('error file not found')

cod_frame=0
inicio = int(round(time.time() * 1000))
detectados = []

veiculos = 0

while cap.isOpened():
    cod_frame+=1
    ret, frame = cap.read()
    if ret == True:

        #BGR -> RGB
        blob = cv2.dnn.blobFromImage(frame, 0.00392,(320,320),(0,0,0),True)
        net.setInput(blob)
        outs = net.forward(outputLayers)

        class_ids ,confidences , boxes = encontrar_objetos_dnn(outs)

        desenha_box(frame, boxes, class_ids, confidences)

        for i in range(len(boxes)):
          x,y,w,h,cx,cy = boxes[i]
          # detectados.append((x,y,w,h,cx,cy))
          veiculos = len(registrar_carro(frame, boxes[i]))
          cv2.circle(frame, (cx,cy) , 4, (0, 0, 255), -1)        

        # for (x,y,w,h,cx,cy) in detectados:
        #   if (390 + 8) > cy > (390 - 8):
        #     carros += 1
        #     detectados.remove((x,y,w,h,cx,cy))
        #     print("Carros detectados: " + str(carros))

        text = f'veiculos: {veiculos}'

        print(text)

        cv2.putText(frame, text, (0, 70), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 5)
        
        # MELHORAR PERFOMANCE, POR ISSO COMENTADO
        # if cod_frame % 25 == 0:
        #   nome_foto = 'foto_'+str(cod_frame)+'.png'
        #   cv2.imwrite(nome_foto, imutils.resize(frame, width=500))
        #   cv2_imshow(cv2.imread(nome_foto))
        #   #cv2_imshow(frame_volume_objetos)
        #   print(cod_frame, boxes, confidences, class_ids)
        
        
        writer.write(frame)

    else:
        break



cap.release()
writer.release()

1280 720
Carros: 2
Carros: 3
Carros: 3
Carros: 3
Carros: 3
Carros: 3
Carros: 3
Carros: 3
Carros: 4
Carros: 4
Carros: 4
Carros: 4
Carros: 4
Carros: 4
Carros: 4
Carros: 4
Carros: 4
Carros: 4
Carros: 4
Carros: 4
Carros: 4
Carros: 4
Carros: 5
Carros: 5
Carros: 5
Carros: 5
Carros: 5
Carros: 5
Carros: 5
Carros: 5
Carros: 6
Carros: 6
Carros: 6
Carros: 6
Carros: 6
Carros: 6
Carros: 6
Carros: 6
Carros: 6
Carros: 6
Carros: 6
Carros: 6
Carros: 6
Carros: 6
Carros: 6
Carros: 6
Carros: 6
Carros: 6
Carros: 6
Carros: 6
Carros: 6
Carros: 6
Carros: 6
Carros: 6
Carros: 6
Carros: 6
Carros: 6
Carros: 6
Carros: 6
Carros: 6
Carros: 6
Carros: 6
Carros: 6
Carros: 7
Carros: 7
Carros: 7
Carros: 7
Carros: 7
Carros: 7
Carros: 8
Carros: 8
Carros: 8
Carros: 8
Carros: 8
Carros: 8
Carros: 8
Carros: 8
Carros: 8
Carros: 8
Carros: 8
Carros: 8
Carros: 8
Carros: 8
Carros: 8
Carros: 8
Carros: 8
Carros: 8
Carros: 8
Carros: 8
Carros: 8
Carros: 8
Carros: 8
Carros: 8
Carros: 8
Carros: 8
Carros: 8
Carros: 8
Carros: 8
Carros: 9
C

In [ ]:
# subtracao = cv2.bgsegm.createBackgroundSubtractorMOG()  # Pega o fundo e subtrai do que está se movendo
# def preprocesso_encontrar_objetos_bordas(frame1):
#   grey = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)  # Pega o frame e transforma para preto e branco
#   blur = cv2.GaussianBlur(grey, (3, 3), 5)  # Faz um blur para tentar remover as imperfeições da imagem
#   img_sub = subtracao.apply(blur)  # Faz a subtração da imagem aplicada no blur
#   dilat = cv2.dilate(img_sub, np.ones((5, 5)))  # "Engrossa" o que sobrou da subtração
#   kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))  # Cria uma matriz 5x5, em que o formato da matriz entre 0 e 1 forma uma elipse dentro
#   dilatada = cv2.morphologyEx(dilat, cv2.MORPH_CLOSE, kernel)  # Tenta preencher todos os "buracos" da imagem
#   dilatada = cv2.morphologyEx(dilatada, cv2.MORPH_CLOSE, kernel)
#   return dilatada

In [ ]:
# def desenha_retangulo_objeto(original, objeto_movimento):
#   centros = []
#   contorno, img = cv2.findContours(objeto_movimento, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#   for (i, c) in enumerate(contorno):
#     (x, y, w, h) = cv2.boundingRect(c)
#     validar_contorno = (w >= 50) and (h >= 50)
#     if not validar_contorno:
#         continue
#     cv2.rectangle(original, (x, y), (x + w, y + h), (0, 255, 0), 2)
#     x1 = w // 2
#     y1 = h // 2
#     cx = x + x1
#     cy = y + y1
#     centros.append((cx,cy))
#   return centros


In [ ]:
# cap = cv2.VideoCapture('./projeto/videos/Car2165.mp4')
# width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# writer = cv2.VideoWriter('./projeto/videos/novo2_Car2165.mp4', cv2.VideoWriter_fourcc(*'DIVX'),20,(width, height))

# print(width, height)
# if cap.isOpened() == False:
#     print('error file not found')

# cod_frame=0

# encontrados = []

# carros = 0

# inicio = int(round(time.time() * 1000))
# while cap.isOpened():
#     cod_frame+=1
#     ret, frame = cap.read()
#     if ret == True:
#         frame_volume_objetos = preprocesso_encontrar_objetos_bordas(frame)
#         centros = desenha_retangulo_objeto(frame, frame_volume_objetos)
        
#         for c in centros:
#           encontrados.append( c )
#           cv2.circle(frame, c, 4, (0, 0, 255), -1)
        
#         for (cx,cy) in encontrados:
#           if (390 + 6) > cy > (390 - 6):
#             carros += 1
#             encontrados.remove((cx,cy))
#             print("Carros encontrados: " + str(carros))

#         text = f'Carros: {carros}'
#         cv2.putText(frame, text, (0, 70), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 5)
        
        
#         if cod_frame % 25 == 0:
#           nome_foto = 'foto_'+str(cod_frame)+'.png'
#           cv2.imwrite(nome_foto, imutils.resize(frame, width=500))
#           cv2_imshow(cv2.imread(nome_foto))
#           cv2_imshow(imutils.resize(frame_volume_objetos, width=500) )
        
        
#         writer.write(frame)

#         time.sleep(float(1 / 25))

#     else:
#         break


# cap.release()
# writer.release()
